#### Импорт необходимых библиотек

In [ ]:
import os
import json
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
import speech_recognition as sr
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
import torch
import logging
from pydub import AudioSegment

#### Создание констант

In [ ]:
# Константы
DATA_DIR = '../data/train/'
DATA_DIR_FILES = [
    'hr_bot_clear',
    'hr_bot_noise',
    'hr_bot_synt'
]
ANNOTATION_DIR = '../data/train/annotation/'
ANNOTATION_FILES = [
    'hr_bot_clear.json',
    'hr_bot_noise.json',
    'hr_bot_synt.json'
]

VAL_DIR = '../data/val/luga/'  # Путь к валидационным данным
ANNOTATION_VAL_FILE = os.path.join(VAL_DIR, 'luga.json')

#### Подготовка данных
Загружаем аудиофайлы и аннотации, которые содержат информацию о командах (текст команды, класс команды, атрибуты). Данные могут быть разделены на чистые (без шума) и зашумленные.

In [ ]:
# Преобразование MP3 в WAV
def convert_mp3_to_wav(mp3_filepath):
    try:
        wav_filepath = mp3_filepath.replace('.mp3', '.wav')
        audio = AudioSegment.from_mp3(mp3_filepath)
        audio.export(wav_filepath, format='wav')
        return wav_filepath
    except Exception as e:
        print(f"Ошибка при конвертации {mp3_filepath} в WAV: {e}")
        return None

# Функция загрузки аннотаций
def load_annotations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# Загрузка датасета с аннотациями
def load_dataset(data_dir, annotation_files):
    dataset = []
    # Перебираем файлы аннотаций
    for annotation_file in annotation_files:
        annotations = load_annotations(os.path.join(ANNOTATION_DIR, annotation_file))

        # Получаем соответствующую директорию на основе названия файла аннотации
        sub_dir = annotation_file.replace('.json', '')
        print(f"Loading from subdirectory: {sub_dir}")

        for item in annotations:
            if all(k in item for k in ['audio_filepath', 'text', 'label', 'attribute']):
                # Используем os.path.join для правильного формирования пути
                audio_path = os.path.join(data_dir, sub_dir, item['audio_filepath'])
                print(f"Checking audio file path: {audio_path}")

                if os.path.exists(audio_path):
                    # Преобразуем MP3 в WAV, если файл в формате MP3
                    if audio_path.endswith('.mp3'):
                        audio_path = convert_mp3_to_wav(audio_path)

                    if audio_path:  # Убедимся, что преобразование прошло успешно
                        dataset.append({
                            'audio_filepath': audio_path,
                            'text': item['text'],
                            'label': item['label'],
                            'attribute': item['attribute']
                        })
                else:
                    print(f"Audio file does not exist: {audio_path}")
            else:
                print(f"Missing keys in item: {item}")
    return dataset

# Загрузка данных
dataset = load_dataset(DATA_DIR, ANNOTATION_FILES)
df = pd.DataFrame(dataset)


#### Извлечение признаков из аудиофайлов
Для того чтобы классифицировать команды, извлекаем признаки из аудиофайлов. Одним из эффективных методов является использование MFCC (Mel-Frequency Cepstral Coefficients).

In [ ]:
# Функция извлечения MFCC признаков
def extract_features(audio_filepath):
    try:
        # Загрузка аудиофайла
        signal, sr = librosa.load(audio_filepath, sr=None)
        # Извлечение MFCC
        mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
        return np.mean(mfccs.T, axis=0)
    except Exception as e:
        print(f"Ошибка при обработке {audio_filepath}: {e}")
        return None

# Извлечение признаков
df['features'] = df['audio_filepath'].apply(extract_features)
df = df.dropna()  # Удаляем строки с ошибками


#### Разделение данных на обучающую и тестовую выборки
После извлечения признаков, разделяем данные на тренировочную и тестовую выборки для обучения модели.

In [ ]:
X = np.array(df['features'].tolist())
y = np.array(df['label'])

# Разделение данных на обучающую и тестовую выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

#### Построение нейронной сети
Создаем нейронную сеть для классификации команд. Модель будет иметь несколько скрытых слоев и выходной слой с количеством нейронов, равным числу классов команд.

In [ ]:
# Определение модели
model = tf.keras.Sequential([
    layers.Input(shape=(13,)),  # Входной слой (13 признаков из MFCC)
    layers.Dense(64, activation='relu'),  # Первый скрытый слой
    layers.Dense(64, activation='relu'),  # Второй скрытый слой
    layers.Dense(23, activation='softmax')  # Выходной слой (23 класса команд)
])

# Компиляция модели
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


#### Обучение модели извлечения текста из аудиофайлов
Обучаем модель на подготовленных данных.

In [ ]:
# Обучение модели
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

#### Оценка модели
Оцениваем качество модели на тестовой выборке для проверки точности распознавания команд.

In [ ]:
# Оценка модели на тестовой выборке
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

#### Сохранение модели
После обучения сохраняем модель для дальнейшего использования.

In [ ]:
# Сохранение модели
model.save('train_command_model.h5')

#### Распознавание речи (speech-to-text)
Для преобразования аудио в текст можно использовать библиотеку speech_recognition или предварительно обученные модели, такие как Wav2Vec2.

In [ ]:
# Функция для распознавания речи
def transcribe_audio(audio_filepath):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_filepath) as source:
        audio = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio)  # Можно использовать оффлайн-модель Wav2Vec2
            print(f"Recognized Text: {text}")
            return text
        except sr.UnknownValueError:
            print("Speech Recognition could not understand audio")
        except sr.RequestError as e:
            print(f"Error in Speech Recognition service: {e}")
    return None


#### Классификация текста команды

In [ ]:
# Классы команд
label = {
    0: "отказ",
    1: "отмена",
    2: "подтверждение",
    3: "начать осаживание",
    4: "осадить на (количество) вагон",
    5: "продолжаем осаживание",
    6: "зарядка тормозной магистрали",
    7: "вышел из межвагонного пространства",
    8: "продолжаем роспуск",
    9: "растянуть автосцепки",
    10: "протянуть на (количество) вагон",
    11: "отцепка",
    12: "назад на башмак",
    13: "захожу в межвагонное,пространство",
    14: "остановка",
    15: "вперед на башмак",
    16: "сжать автосцепки",
    17: "назад с башмака",
    18: "тише",
    19: "вперед с башмака",
    20: "прекратить зарядку тормозной магистрали",
    21: "тормозить",
    22: "отпустить",
}

# Функция для классификации команды
def classify_command(text):
    for command in _label.values():
        if command in text:
            return command
    return "Unknown Command"


#### Проверка точности на реальных данных
Проверяем работу системы на новых аудиофайлах и оцениваем точность распознавания и классификации.

In [ ]:
def evaluate_commands():
    annotations = load_annotations(ANNOTATION_FILE)
    correct_predictions = 0
    total_predictions = len(annotations)

    # Словарь для хранения статистики
    statistics = {
        'correct': {},
        'total': {},
        'accuracy': {}
    }

    for item in annotations:
        audio_filepath = item['audio_filepath']
        label = item['label']

        audio_path = os.path.join(VAL_DIR, audio_filepath)
        if os.path.exists(audio_path):
            prediction = predict_command(audio_path)
            print(f'Predicted: {prediction}, Actual: {label} for {audio_path}')

            if prediction == label:
                correct_predictions += 1

    accuracy = correct_predictions / total_predictions

    # Сохраняем точность в статистике
    statistics['accuracy'] = accuracy

    # Выводим статистику
    print(f'\nTotal Predictions: {total_predictions}')
    print(f'Correct Predictions: {correct_predictions}')
    print(f'Accuracy: {accuracy:.2f}')
    print('Detailed statistics per label:')
    for label in statistics['total']:
        total = statistics['total'][label]
        correct = statistics['correct'].get(label, 0)
        print(f'Label: {label}, Total: {total}, Correct: {correct}, Accuracy: {correct / total:.2f}' if total > 0 else f'Label: {label}, Total: {total}, Correct: {correct}')


evaluate_commands()